In [1]:
import gzip
import pandas as pd
import numpy as np
import os

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from collections import defaultdict
import math
import scipy.optimize
import numpy
import string
import random
from sklearn import linear_model

In [2]:
def parse(f):
    for l in gzip.open(f):
        yield eval(l)

In [3]:
dataset = list(parse(os.path.join('data', "trainRecipes.json.gz")))

In [4]:
train = dataset[:150000]
valid = dataset[150000:175000]
test = dataset[175000:]

In [ ]:
MOdel 

In [20]:
ingsPerItem = defaultdict(set)
itemsPerIng = defaultdict(set)

In [21]:
for d in dataset:
    r = d['recipe_id']
    for i in d['ingredients']:
        ingsPerItem[r].add(i)
        itemsPerIng[i].add(r)

In [17]:
def Jaccard(s1, s2):
    num = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return num / denom

In [22]:
def mostSimilar(i, N):
    ings = ingsPerItem[i]
    similarities = []
    for each in ingsPerItem:
        if i == each: continue
        sim = Jaccard(ings, ingsPerItem[each])
        similarities.append((sim, each))
    similarities.sort(reverse=True)
    return similarities[:N]

In [23]:
mostSimilar('06432987', 5)

[(0.4166666666666667, '68523854'),
 (0.38461538461538464, '12679596'),
 (0.36363636363636365, '79675099'),
 (0.36363636363636365, '56301588'),
 (0.35714285714285715, '87359281')]

In [24]:
def mostSimilarIng(i, N):
    items = itemsPerIng[i]
    similarities = []
    for each in itemsPerIng:
        if i == each: continue
        sim = Jaccard(items, itemsPerIng[each])
        similarities.append((sim, each))
    similarities.sort(reverse=True)
    return similarities[:N]

In [26]:
mostSimilarIng('butter', 5)

[(0.22315311514274808, 'salt'),
 (0.2056685424969639, 'flour'),
 (0.19100394157199166, 'eggs'),
 (0.17882420717656095, 'sugar'),
 (0.17040052045973944, 'milk')]

In [29]:
recipes = pd.DataFrame(dataset)

To pull recipes that capture the "essence" of the set of ingredients, I think it would be more precise to focus on special ingredients in the set, and to put less priority on basic ingredients. Then, we could determine the Jaccard Similarities of pairs of ingredients in the set, and pull ingredients based on the weighted average of the ingredients, with more rare ingredients weighing more than common ones.

In [237]:
def essence(ingset):
    # creates a dataframe with ingredients as the index and each one's "rarity" or frequency in recipes
    rarity = pd.Series([each for every in list(recipes['ingredients']) for each in every]).value_counts().sort_values()
    ingredients = pd.DataFrame()
    ingredients['ingredient'] = test
    ingredients['#'] = ingredients['ingredient'].apply(lambda x: 0 if x not in rarity else rarity[x])
    # removes ingredients that are not in the ingredient list
    ingredients = ingredients.replace(0, pd.np.nan).dropna(axis=0, how='any')
    # pulls the 5 most "rare" ingredients to remove basic and common ingredients
    ingredients = ingredients.loc[:4,]
    ingredients['weight'] = ingredients['#'].apply(lambda x: (x /sum(ingredients['#'])))
    ingredients = ingredients.sort_values('weight')
    # performs Jaccard Similarity with 5,4,3,2,1 items as a result, corresponding to its ranking in rarity
    num_pulls = [5,4,3,2,1]
    ingredients['# of pulls'] = num_pulls[:ingredients.shape[0]]
    
    def mostSimilarIng(i, N):
        items = itemsPerIng[i]
        similarities = []
        for each in itemsPerIng:
            if i == each: continue
            sim = Jaccard(items, itemsPerIng[each])
            similarities.append((sim,each))
        similarities.sort(reverse=True)
        return similarities[:N]
    
    # creates dataframe pairing ingredient with its similar ingredients
    similar = pd.DataFrame(ingredients['ingredient'])
    similar['similar'] = [mostSimilarIng(ingredients['ingredient'][each], ingredients['# of pulls'][each]) for each in ingredients.index]
    similar['match'] = similar['similar'].apply(lambda x: [each[1] for each in x])
    # combines all sourced ingredients with inputted ingredients as one list to search for in recipes
    curated_list = list(similar['match'].explode()) + ingset
    
    def mostSimilarRec(i, N):
        ings = set(i)
        similarities = []
        for each in ingsPerItem:
            sim = Jaccard(ings, ingsPerItem[each])
            similarities.append((sim, each))
        similarities.sort(reverse=True)
        return similarities[:N]

    recipe_list = mostSimilarRec(curated_list, 5)
    return [recipes.loc[recipes['recipe_id'] == each[1], 'name'].iloc[0] for each in recipe_list]

In [236]:
test = ["cinnamon", "cherries", "butterscotch", "vodka"]
results = essence(test)
results

/tmp/ipykernel_278/2179408309.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  ingredients = ingredients.replace(0, pd.np.nan).dropna(axis=0, how='any')


['4th of july martini',
 'beautiful blue vodka martini',
 'shark bite',
 'barbados sunrise',
 'halloween black brew']